In [ ]:
#import libraries and set up display settings of notebook
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn'
import datetime
import glob
import os
import json

In [ ]:
SCRIPT_PATH = os.path.abspath('')
#SCRIPT_PATH = os.path.dirname(os.path.abspath(__file__))
#LPG = os.path.join(SCRIPT_PATH, '..', 'resources', 'LPG.txt')
LPG = os.path.join(SCRIPT_PATH, 'LPG.txt')
with open(LPG, 'r') as file:
    DATA = json.load(file)

AL_LOGO = os.path.join(SCRIPT_PATH, 'ALlogo.png')

In [ ]:
DATA['units']
list(DATA['units'].keys())

# Data analysis functions

In [ ]:
def classify_cols(selected):
    print("classify_cols started ---")
    print(f"{selected=}")

    units = DATA['units']

    filter_unit_cols = {col : unit for col, unit in units.items() if col in selected}
    # {key_expression: value_expression for item in iterable if condition}
    print(f"{filter_unit_cols=}")

    classified_cols = {}
    for col, unit in filter_unit_cols.items():
        if unit not in classified_cols:
            classified_cols[unit] = [] #if first column of this unit, create key and an empty array as value
        classified_cols[unit].append(col)
    
    print(f"{classified_cols=}")
    return classified_cols

# Plot functions

In [ ]:
import pandas as pd
from PIL import Image
import datetime
import os
import json
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Alfa Laval brand colors
ALcolors = ['rgba(17, 56, 127, 1)', #AL blue
            'rgba(0, 0, 0, 1)', #AL white
            'rgba(220, 146, 118, 1)', #AL earth
            'rgba(254, 205, 96, 1)', #AL sun
            'rgba(147, 199, 198, 1)', #AL water
            'rgba(0, 127, 200, 1)', #AL innovation
            ]

In [ ]:
def custom_alarms(x, y, cat, labels):
    marker_symbols = ['circle', 'square', 'diamond', 'cross', 'x']  # Define marker symbols for different values
    
    # Determine marker symbol based on the value of y
    if y == 'value1':
        marker_symbol = marker_symbols[0]
    elif y == 'value2':
        marker_symbol = marker_symbols[1]
    elif y == 'value3':
        marker_symbol = marker_symbols[2]
    elif y == 'value4':
        marker_symbol = marker_symbols[3]
    else:
        marker_symbol = marker_symbols[4]  # Default to the last marker symbol
    
    # Create the trace with the determined marker symbol
    trace = go.Scatter(
        x=x,
        y=y.astype(str),
        name=cat,
        mode='markers',
        marker_symbol=marker_symbol,
        marker_line_color=ALcolors[2],
        marker_color=ALcolors[3],
        marker_line_width=1,
        marker_size=8,
        legendgroup="Alarms",
        legendgrouptitle_text="Alarms",
        hovertext=labels
    )
    
    return trace


def plot_alarms(x,y,labels,cat):
    trace = go.Scatter(x=x, y=y.astype(str),
                       name=cat,
                       mode='markers', marker_symbol='x',
                       marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                       marker_line_width=1, marker_size=8,
                       legendgroup="Alarms",legendgrouptitle_text="Alarms",
                       hovertext=labels)
    return trace

def plot_events(x,y,labels):
    trace = go.Scatter(x=x, y=y.astype(int),
                       name='Event Number',
                       mode='markers',marker_symbol='star',
                       marker_line_color=ALcolors[5], marker_color=ALcolors[5],
                       marker_line_width=1, marker_size=8,
                       legendgroup="Events",legendgrouptitle_text="Events",
                       hovertext=labels)
    return trace

def line_trace(x,y,name,cat,color=None):
    if color is not None:
        trace = go.Scatter(x=x, y=y,
                           name = name,
                           line=dict(color = color), line_shape='spline',
                           legendgroup=cat, legendgrouptitle_text=cat)    
    else: #default plotly color assignment
        trace = go.Scatter(x=x, y=y,
                           name = name,
                           line_shape='spline',
                           legendgroup=cat, legendgrouptitle_text=cat)
    return trace

def square_line_trace(x,y,name,cat,color=None,labels=None):
    if color is not None:
        trace = go.Scatter(x=x, y=y,
                           name = name,
                           line=dict(color = color), line_shape='hv',
                           legendgroup=cat, legendgrouptitle_text=cat)
    else: #default plotly color assignment
        trace = go.Scatter(x=x, y=y,
                   name = name, line_shape='hv',
                   legendgroup=cat, legendgrouptitle_text=cat)

    if isinstance(labels, pd.Series):
        trace.hovertext = labels
    
    return trace

def square_disc_line_trace(x,y,name,color,cat):
    trace = go.Scatter(x=x, y=y,
                       name = name,
                       line=dict(color = color, dash='dot'), line_shape='hv',
                       legendgroup=cat, legendgrouptitle_text=cat)
    return trace

def filled_trace(x,y,name,color,cat):
    trace = go.Scatter(x=x,y=y,
                       name= name,
                       fill='tozeroy', mode='none', 
                       fillcolor = color,
                       line_shape='hv')
    if cat is not None:
        trace.legendgroup = cat
        trace.legendgrouptitle_text = cat
    
    return trace

In [ ]:
def custom_plot_divided(dfs, dfa, cols, date1, date2, tittle): # n rows, one for each unit
    print("custom_plot1 started ---")


    # Columns with its respective unit
    units = DATA['units']

    print("date limits:")
    print(f"{date1=},{date2=}")
    
    # filter dataframes for date interval selected
    if not dfs.empty:
        dfs = dfs[(dfs['Timestamp'] >= date1) & (dfs['Timestamp'] <= date2)]
    else:
        dfs = pd.DataFrame()
        print("Standard Logs empty in date range selected")

    if not dfa.empty:
        dfa = dfa[(dfa['Timestamp'] >= date1) & (dfa['Timestamp'] <= date2)]
    else:
        dfa = pd.DataFrame()
        print("Alarm Logs empty in date range selected")

    #returns an empty plot if no data to plot
    if dfs.empty and dfa.empty:
        fig = go.Figure()
        fig.update_layout(title_text=tittle)
        print("--- no data to plot")
        return fig

    # From column list, get a dictionary with cols classified by unit type
    cols2 = classify_cols(cols)
    print("selected units and columns:")
    print(cols2)

    print("fig init")
    fig = make_subplots(rows=len(cols2), cols=1, shared_xaxes=True, vertical_spacing=0.02)

    # Iterate classified cols and create traces
    for i, (unit, cols) in enumerate(cols2.items()):
        print(f"{i=}, {unit=}, {cols=}")

        fig.update_yaxes(title_text=unit,row=i+1)

        for col in cols:
            print(f"{col=}")

            if not dfs.empty:
                    # If unit is a bool, int or valve_pos(int), create a square line trace instead of spline
                    if unit in ['bool', 'int', 'valve_pos']:
                        trace = square_line_trace(
                            x=dfs['Timestamp'],
                            y=dfs[col],
                            name=col,
                            cat=unit)
                    else:
                        trace = line_trace(
                            x=dfs['Timestamp'],
                            y=dfs[col],
                            name=col,
                            cat=unit)

                    fig.add_trace(trace, row=i+1, col=1)

    print("fig config")

    # Update layout properties
    fig.update_layout(hovermode="x unified", hoverlabel=dict(bgcolor='rgba(255,255,255,0.75)', namelength = -1, font=dict(color='black')),  
        legend=dict(groupclick="toggleitem"), #avoid grouping all traces
        title_text=tittle , title_x=0.5
    )

    # Add image
    alLogo = Image.open(AL_LOGO)
    fig.add_layout_image(
        dict(
            source=alLogo,
            xref="paper", yref="paper",
            x=0, y=1.025,
            sizex=0.14, sizey=0.14,
            xanchor="left", yanchor="bottom"
        )
    )
    
    print("fig done")
    return fig

In [ ]:
def plot_alarms(x,y,labels,cat):
    trace = go.Scatter(x=x, y=y.astype(str),
                       name=cat,
                       mode='markers', marker_symbol='x',
                       marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                       marker_line_width=1, marker_size=8,
                       legendgroup="Alarms",legendgrouptitle_text="Alarms",
                       hovertext=labels)
    return trace

# LPG

In [ ]:
foldername = "logs/ProcessLogs/"
# Define files path
Path = "ProcessLog?*.csv"

# Read all the files name in the directory
AllFilesname = glob.glob(foldername + Path)

# Create an empty list to store the dataframes
ListDataframe = list()

# Total file number
NoFiles = len(AllFilesname)

# For each file in the specified directory import the data and add it in the list
for Filename in AllFilesname:
    ListDataframe.append(pd.read_csv(Filename, sep=';', skiprows=3, decimal=',', encoding='unicode_escape'))

# Concatenate the files in the list in unique dataframe
DataImport = pd.concat(ListDataframe, axis=0, ignore_index=True)

# Change datatypes
DataImport['Timestamp'] = pd.to_datetime(DataImport['Timestamp'], format='%Y %m %d %H:%M:%S:%f')
# ordering ascending
DataImport = DataImport.sort_values(by='Timestamp', ascending=True)


print (DataImport.shape)
DataImport.tail()

In [ ]:
Filename = 'logs/AlarmHistory/Alarms_2023_11_06_00_32_23.csv'

alarmhist = pd.read_csv(Filename, sep=',', decimal='.', encoding='unicode_escape')

# Rename Time
alarmhist.rename(columns = {'Time':'Timestamp',' Change':'Change',
                        ' Instance':'Instance', ' Name':'Name',
                         ' Code':'Code', ' Severity':'Severity',
                         ' AdditionalInformation1':'AdditionalInformation1',
                         ' AdditionalInformation2':'AdditionalInformation2',
                         ' Message':'Message'}, inplace = True)


# Change datatypes
alarmhist['Timestamp'] = pd.to_datetime(alarmhist['Timestamp'], format='%Y-%m-%d %H:%M:%S:%f')
# ordering ascending
alarmhist = alarmhist.sort_values(by='Timestamp', ascending=True)
# translate values
alarmhist['Change_val'] = alarmhist['Change'].replace({' Active -> Inactive' : '0',
                                                   ' Inactive -> Active' : '1',
                                                   ' Unacknowledged -> Acknowledged': '2'})
alarmhist['Change_val'] = alarmhist['Change_val'].astype(int)

# Show results
print (alarmhist.shape)
alarmhist.head()

In [ ]:
# Pivot the DataFrame
pivot_df = alarmhist.pivot(index='Timestamp', columns='Name', values='Change')

# Reset index to have Timestamp as a column
pivot_df = pivot_df.reset_index()

pivot_df.tail()

In [ ]:
# Create an empty list to store traces
traces = []

# Iterate over the columns of the pivot DataFrame
for column in pivot_df.columns[1:]:
    # Create a DataFrame for each column without NaN values
    df_col = pivot_df[['Timestamp', column]].dropna()
    
    # Create a trace for the current column
    trace = go.Scatter(x=df_col['Timestamp'], y=df_col[column], line_shape='hv', name=column)
    #line_shape='hv'
    
    # Append the trace to the list of traces
    traces.append(trace)

# Create the figure
fig = go.Figure(data=traces)

# Update layout if needed
fig.update_layout(title="Scatter Plot for Each Column Without NaN Values")

# Show the figure
fig.show()

In [ ]:
# Create an empty list to store traces
traces = []
marker_symbols = {0: 'circle', 1: 'square', 2: 'diamond'}
marker_lcolors = {0: 'green', 1: 'red', 2: 'yellow'}
marker_colors = {0: 'green', 1: 'red', 2: 'yellow'}

# Iterate over each unique column (Name)
for col in pivot_df.columns[1:]:
    # Filter out NaN values and create a DataFrame for each unique value (0, 1, 2)
    for val in [0, 1, 2]:
        filtered_df = pivot_df[pivot_df[col] == val].dropna(subset=[col])
        # Create scatter plot trace for each value

        trace = go.Scatter(x=filtered_df['Timestamp'], y=filtered_df[col],
                        name=f'{col} - {val}',
                        mode='markers', marker_symbol=marker_symbols[val],
                        marker_line_width=1, marker_size=8)
                        #legendgroup="Alarms",legendgrouptitle_text="Alarms")
        
        # Append the trace to the list of traces
        traces.append(trace)

# Create the figure
fig = go.Figure(data=traces)

# Update layout if needed
fig.update_layout(title="Scatter Plot for Each Column Without NaN Values")

# Show the figure
fig.show()

In [ ]:
print(classify_cols(["FIT1001 - Flow signal for the client FIT1001","FV1001A - Regulating valve position command  FV100","P1001A - LP Pump - Speed command P1001A_SPD_CMD","P4001A- GW Pump - Start command P4001A_STR_CMD"]))

In [ ]:
custom_plot_divided(DataImport, pd.DataFrame(), ["FIT1001 - Flow signal for the client FIT1001","FV1001A - Regulating valve position command  FV100","P1001A - LP Pump - Speed command P1001A_SPD_CMD","P4001A- GW Pump - Start command P4001A_STR_CMD"], '2023-09-18', '2023-09-19', 'tittle').show()